## Simulating the data

In [13]:
import numpy as np

zeros = [100, 90, 80, 70, 60, 50]
for zero_amount in zeros:
    myvar = np.random.choice([0, 1], size=8*10**8, replace=True, p=[zero_amount/100, 1-zero_amount/100])
    myvar = np.packbits(myvar) 
    f = open("zeros_%dp" % zero_amount, "wb")
    f.write(myvar)
    f.close()

In [12]:
# zero_amount = 100
# myvar = np.random.choice([0, 1], size=8*10**8, replace=True, p=[zero_amount/100, 1-zero_amount/100])
# myvar = np.packbits(myvar) 
# f = open("zeros_%dp" % zero_amount, "wb")
# f.write(myvar)
# f.close()

In [14]:
nt_seq = np.random.choice(['A', 'T', 'C', 'G'], size=10**8)
f = open("nt_seq.fa", "w")
f.write("".join(nt_seq))
f.close()

In [15]:
protein_seq = np.random.choice(['G', 'A', 'L', 'M', 'F', 'W', 'K', 'Q', 'E', 'S', 'P', 'V', 'I', 'C', 'Y', 'H', 'R', 'N', 'D', 'T'], size=10**8)
f = open("protein_seq.fa", "w")
f.write("".join(nt_seq))
f.close()

### Commands

time gzip -c nt_seq.fa > nt_seq.fa.gz

time bzip2 protein_seq.fa -fk

time pbzip2 protein_seq.fa -fk

| File name      | Original File size | Time to gzip, size | Time to bzip2, size | Time to pbzip2, size |
|----------------|--------------------|--------------------|---------------------|----------------------|
|   zeros_100p   |         96M        |     0.594s, 96K    |     0.147s, 4.0K    |     0.140s, 8.0K     |
|    zeros_90p   |         96M        |    26.417s, 54M    |     9.799s, 56M     |      1.051s, 56M     |
|    zeros_80p   |         96M        |    17.029s, 74M    |     10.256s, 79M    |      1.219s, 79M     |
|    zeros_70p   |         96M        |     6.963s, 86M    |     11.122s, 91M    |      1.453s, 91M     |
|    zeros_60p   |         96M        |     4.318s, 94M    |     12.919s, 96M    |      1.724s, 96M     |
|    zeros_50p   |         96M        |     3.490s, 96M    |     12.883s, 96M    |      1.856s, 96M     |
|    nt_seq.fa   |         96M        |    23.584s, 28M    |     9.527s, 27M     |      0.998s, 27M     |
| protein_seq.fa |         96M        |    23.592s, 28M    |     9.550s, 27M     |      1.027s, 27M     |

- Which algorithm achieves the best level of compression on each file type?  
The algorithm which achieves the best compression on zeros is generally gzip, although the file for all zeros is compressed best by bzip2. For the nucleotide sequence, the algorithm which performs the best is a tie between bzip2 and pbzip2. This is also true of the amino acid sequence.
- Which algorithm is the fastest? What is the difference between bzip2 and pbzip2? Do you expect one to be faster and why?  
The algorithm which is the fastest is pbzip2. This makes sense because it applies a fast transform but uses parallelization, which allows for concurrent work to be done. The difference between bzip2 and pbzip2 is that pbzip2 is bzip2 run in parallel. This means that pbzip2 should run faster because it is using resources concurrently instead of linearly.
- How does the level of compression change as the percentage of zeros increases? Why does this happen?   
The level of compression increases as the percentage of zeros increases because it is easier to represent many repeated values with compression, since there is a lower amount of entropy and therefore less bits needed to represent the data.
- What is the minimum number of bits required to store a single DNA base?  
The minimum number of bits required to store a single DNA base is 2. This is because there are 4 possible states for the DNA base to be, which means that only 2 bits are needed to represent the base.
- What is the minimum number of bits required to store an amino acid letter?  
The minimum number of bits required to store an AA letter is 5, which will be able to store 32 possible values. There are 20 amino acids, and using 4 or fewer doesn't allow us to represent at least 20 amino acids.
- In your tests, how many bits did gzip and bzip2 actually require to store your random DNA and protein sequences?   
gzip and bzip2 required 27M to store the DNA sequence, which is 2.7\*8 bits per DNA base, or 21.6 bits. This may be because the compression tries to preserve some idea of the full representation of the characters. This is also true for the proteins.
- Are gzip and bzip2 performing well on DNA and proteins?   
They are performing okay on DNA and proteins. I think there is some problems based on what was discussed above.

### gp120

I pasted all the sequences that I found into text files which are stored in a folder. Then I concatenated them into a single file.

In [23]:
sequence = ""
for i in range(1, 11):
    with open("gp120/%d.txt" % i, "r") as f:
        for line in f:
            sequence += line.replace("\n", "")

I expect better compression because the sequence is not completely random.

In [24]:
f = open("gp120_seq.fa", "w")
f.write(sequence)
f.close()

Compressing the sequences with pbzip2 takes 0.006s and compresses from 44K to 4.0K, which is a factor of 11x.

Compressing the sequences with bzip2 takes 0.004s and compresses from 44K to 4.0K, which is a factor of 11x. The reason why it is faster than pbzip2 is because of the extra overhead time taken to do the parallelization.

The compressing is much better than random data, which is around 3x compression.

## Estimating compression of 1000 terabytes

Since 80% is compressible by 11x, 10% is compressible by 3x, and 10% is compressible by 1x (completely random, i.e. not compressible). This is an overall reduction of from 1.0 to 0.206, or 4.85x.

Since we save 20.6 percent of the space, we save 206 terabytes. This means we get a bonus of 20.6 * 500, or 10300 in savings.